Shortest Distance Query

For more information about the project, please refer to the [project specification](https://cgi.cse.unsw.edu.au/~cs9312/24T2/project/). You can edit this file and add anything you like. We will only use the code cell of the `ShortestDistanceQuery` class for testing. You can add descriptions and some theoretical analysis (e.g. index space, query time complexity, and index time complexity) to this file without creating a separate PDF document.

**Note**: Make sure to **sequentially run all cells in each section** in order to carry over intermediate variables/packages to the next cell.

## 1. Code Template
You need to implement the `ShortestDistanceQuery` class to support shortest distance queries for large graphs (e.g. hundreds of millions of vertices and edges). A code template is given below.

The `ShortestDistanceQuery` class is initialized by a graph `G`. The data structure of `G` will be presented in the next section. The class calls the function `preprocess` to precompute some index structure for `G`. The `query` function has two inputs: `vertex_u` and `vertex_v`. The function outputs the shortest distance between `vertex_u` and `vertex_v` in `G`.

In [5]:
################################################################################
# You can import any Python Standard Library modules~
from collections import deque
import copy
import numpy as np
from tqdm import tqdm
################################################################################

class ShortestDistanceQuery(object):
    def __init__(self, G):
        self.G = G

        self.adj_list = G.adj_list

        self.L = [set() for _ in range(G.vertex_num)]
        self.L_dist = [{} for _ in range(G.vertex_num)]

        self.processed = np.zeros(G.vertex_num, dtype=bool)

        # Calculate degree of each vertex
        self.degrees = np.array([len(self.adj_list[v]) for v in range(G.vertex_num)])

        self.preprocess(G)

    def preprocess(self, G):
        
        #sort the vertices by degree
        vertices = np.argsort(-self.degrees)


        for u in tqdm(vertices):
            visited = np.copy(self.processed)

            self.L[u].add(u)
            self.L_dist[u][u] = 0
            queue = deque()

            queue.append((u,0))

            #BFS
            while queue:
                (curr_u,dist_u) = queue.popleft()
                visited[curr_u] = True
                for w in self.adj_list[curr_u]:
                    if visited[w]:
                        continue
                    if w not in self.L_dist[u] or self.L_dist[u][w] > dist_u + 1:
                        self.L[w].add(u)
                        self.L_dist[w][u] = dist_u+1
                        self.L[u].add(w)
                        self.L_dist[u][w] = dist_u+1
                    queue.append((w,dist_u+1))

            self.processed[u] = True   
        
        return
    
    
    def query(self, vertex_u, vertex_v):
        shortest_distance = 0

        if vertex_u == vertex_v:
            return 0
        
        if vertex_v in self.adj_list[vertex_u]:
            return 1
        
        intersect_value = self.intersect(self.L[vertex_u],self.L[vertex_v])
        
        if len(intersect_value) == 0:
            return -1
        
        for u in intersect_value:
            dist = self.L_dist[vertex_u][u] + self.L_dist[u][vertex_v]
            if shortest_distance == 0 or dist < shortest_distance:
                #print(f"Update shortest_distance: {dist} with u: {u}")
                shortest_distance = dist
        
        return shortest_distance

    def intersect(self,L1,L2):
        return set(L1) & set(L2)
        

## 2. Graph Data Structure
The following is the data stucture of the input graph `G`.

In [6]:
class UndirectedUnweightedGraph(object):
    def __init__(self, edge_list):
        self.adj_list = []
        self.vertex_num = 0
        self.edge_num = 0
        info = True
        for [vertex_u, vertex_v] in edge_list:
            if info:
                info = False
                self.vertex_num = vertex_u
                self.edge_num = vertex_v
                self.adj_list = [list() for _ in range(self.vertex_num)]
            else:
                self.adj_list[vertex_u].append(vertex_v)
                self.adj_list[vertex_v].append(vertex_u)

## 3. How to test your code

### 3.1 Download the sample dataset.

Running the following command will create the **COMP9312-24T2-Project** folder, which contains the files for the three datasets.

> **Cora** (2k vertices) is a real citation graph, **map_BJ_part** (4k vertices) is a real road network for a small area of Beijing, and **map_NY_part** (7k vertices) is a real road network for a small area of New York. For the two road networks, we erase the weight of each edge in the original dataset to generate unweighted graphs for simplicity.

There are three files for each dataset, where ***.graph** includes the graph information and all graph edges (vertex IDs are consecutive and start from 0), ***.query** includes a set of queries for testing. ***.answer** includes the correct answer to each query for your reference.

If the dataset already exists, an error like "*destination path 'COMP9312-24T2-Project' already exists*" will appear.

**NOTE**: We will test the code using different datasets.

In [ ]:
!git clone https://github.com/kevinChnn/COMP9312-24T2-Project

### 3.2 The main function

Our test procedure first loads the graph dataset and the query dataset. Then, it calls the `ShortestDistanceQuery` class to preprocess the graph. After that, it will run each query and test its efficiency and correctness.

In [7]:
import time
import numpy as np

if __name__ == "__main__":

    print('\n######## Loading the dataset...')
    # edge_list = np.loadtxt('./sample_test.graph', dtype=int)
    # query_list = np.loadtxt('./sample_test-sd.query', dtype=int)
    # edge_list = np.loadtxt('./COMP9312-24T2-Project/cora.graph', dtype=int)
    # query_list = np.loadtxt('./COMP9312-24T2-Project/cora-sd.query', dtype=int)
    edge_list = np.loadtxt('./COMP9312-24T2-Project/map_BJ_part.graph', dtype=int)
    query_list = np.loadtxt('./COMP9312-24T2-Project/map_BJ_part-sd.query', dtype=int)
    # edge_list = np.loadtxt('./COMP9312-24T2-Project/map_NY_part.graph', dtype=int)
    # query_list = np.loadtxt('./COMP9312-24T2-Project/map_NY_part-sd.query', dtype=int)
    G = UndirectedUnweightedGraph(edge_list)
    print("Number of vertices:", G.vertex_num)
    print("Number of edges:", G.edge_num)
    print("Edge list:", G.adj_list)

    print('\n######## Preprocessing the graph...')
    start_preprocessing = time.time()
    SDQ = ShortestDistanceQuery(G)
    end_preprocessing = time.time()
    print("Preprocessing time: {:.8f}".format(end_preprocessing-start_preprocessing))

    print('\n######## Query Testing...')
    start_query = time.time()
    count_false = 0
    for i in range(query_list.shape[0]):
        distance = SDQ.query(query_list[i][0], query_list[i][1])
        print("The shortest distance between {} and {} is: (output = {}, expected = {}) {}".format(query_list[i][0], query_list[i][1],distance, query_list[i][2], query_list[i][2]==distance))
        if query_list[i][2]!=distance:
            print("FOUND FALSE! The shortest distance between {} and {} is: (output = {}, expected = {})".format(query_list[i][0], query_list[i][1],distance, query_list[i][2]))
            count_false += 1
            break
    end_query = time.time()
    print("Count of false outputs:", count_false)
    print("Average query time: {:.8f}".format((end_query-start_query)/query_list.shape[0]))


######## Loading the dataset...
Number of vertices: 4021
Number of edges: 4812
Edge list: [[1, 2], [0, 692, 1176, 2773], [0, 1782], [4, 5, 6], [3, 1432, 1895], [3, 6, 2498, 2499], [3, 5, 1948, 2766], [8, 9], [7, 362, 687, 3285], [7, 352, 353], [11, 12, 13, 14, 15], [10, 12, 678, 679], [10, 11, 424, 2057], [10, 286], [10, 423, 424, 425], [10, 423, 432, 2042], [17, 18, 19], [16, 545, 547, 548], [16, 553, 3834, 3835], [16, 35, 1206], [21, 22, 23], [20, 1492, 1728], [20, 884, 885], [20, 73], [25, 26, 27, 28, 29], [24, 1360, 3806, 3822], [24, 27, 3680, 3824], [24, 26, 1877, 3817], [24, 638, 3808, 3812], [24, 1712, 3814, 3819], [31, 32, 33, 34], [30, 631], [30, 630, 1831], [30, 640, 1595], [30, 2511], [19, 36, 37, 38], [35, 544, 545, 546], [35, 1212, 1214], [35, 1234, 1319], [40, 41, 42, 43, 44], [39, 1856], [39, 647, 1855], [39, 1516, 1577], [39, 2929], [39, 347, 3211], [46, 47, 48, 49], [45, 929, 2108, 2109, 2110], [45, 1405, 1729, 1730], [45, 88, 464], [45, 87, 1729, 2109], [51, 52, 53],

100%|██████████| 4021/4021 [00:03<00:00, 1067.92it/s]


Preprocessing time: 3.79660320

######## Query Testing...
The shortest distance between 729 and 2682 is: (output = 26, expected = 26) True
The shortest distance between 962 and 1522 is: (output = 12, expected = 12) True
The shortest distance between 1606 and 310 is: (output = 23, expected = 23) True
The shortest distance between 577 and 1452 is: (output = 26, expected = 26) True
The shortest distance between 718 and 3189 is: (output = 28, expected = 28) True
The shortest distance between 579 and 1485 is: (output = 27, expected = 27) True
The shortest distance between 2173 and 460 is: (output = 43, expected = 43) True
The shortest distance between 1811 and 1234 is: (output = 27, expected = 27) True
The shortest distance between 1108 and 366 is: (output = 7, expected = 7) True
The shortest distance between 2371 and 3243 is: (output = 31, expected = 31) True
The shortest distance between 2856 and 3271 is: (output = 51, expected = 51) True
The shortest distance between 1306 and 1880 is: (o